## Automated Extraction of Leaf Area from Photos

##### Take photos of leaves with any camera*, populate a folder, and quickly extract leaf data

*on white background of known dimensions

To answer some important plant biology, horticultural or agronomic research questions it can be important to accurately measure the area of many leaves. This labor intensive process has previously limited the size of datasets used in publications, and a faster solution would vastly improve the size and quality of these datasets.

The following code represents a fast, reliable and simple framework for rapidly extracting leaf area from large numbers of images captured with a camera.

Make sure you set the name of the __image directory__ correctly, and assign the __background__ with the correct size, using desired units.

In the final cell, you may set a __destination directory__ and __filename__ for exporting .csv file of all leaf area data.

In [1]:
# import packages
import os
import cv2
import skimage
import pandas as pd
import imutils
from imutils.perspective import four_point_transform
from PIL import Image

In [2]:
# define function for getting list of image files in user provided directory
def getListOfFiles(dirName):
    # create a list of file and sub directories names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

### Set some variables

1) Edit the __folder name__ and __paper size__ to appropriate values.

2) Set paper or white background size using any units you prefer.

In [9]:
# set folder name (including path from this jupyter notebook)
folder = "photos" # (edit as needed)

# set paper or white background size
paper_size = 0.06032246 # Standard USA Letter paper, m^2 (edit as needed)

# build list of image files (do not edit)
file_list = getListOfFiles(folder)
if 'photos/.DS_Store' in file_list:
    file_list.remove('photos/.DS_Store')

### Loop over all the photos in folder

Extract leaf area from each image, then add file name, date of capture and leaf area to a dataframe.

In [10]:
# reset counter
z = 0

# loop over all images in the list of image files
while z < len(file_list):

    # load the image
    image = cv2.imread(file_list[z])

    # resize image
    image = imutils.resize(image, height = 1000)

    # convert resized image to grayscale, blur it slightly, then find edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    
    # apply Otsu's thresholding method to create a binarized version
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    # find contours in the edge map, keeping only the largest ones
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True) # approximate the contour
        if len(approx) == 4: # if approx contour has four points, then assume that we found paper
            screenCnt = approx
            break
            
    # apply the four point transform to obtain a top-down view of the paper area only         
    warped = four_point_transform(thresh, screenCnt.reshape(4, 2))  

    # extract leaf area
    n_white = cv2.countNonZero(warped)
    height, width = warped.shape
    n_total = height * width
    leaf_area = (n_white / n_total) * paper_size

    # extract creation time from image metadata
    imageName = file_list[z]
    creation_time = Image.open(imageName)._getexif()[36867]

    if z == 0:

        # Create the pandas DataFrame
        dict = {'File Name':[],'Date Time':[],'Leaf Area':[]}
        df = pd.DataFrame(dict)
        
        z += 1

    else:
        # append onto existing results
        dict = {'File Name': [imageName[7:20]],'Date Time': [creation_time[:19]], 'Leaf Area': [leaf_area]}
        new_row = pd.DataFrame(dict)
        df = pd.concat([new_row, df], ignore_index = True)

        z += 1

### Print the dataframe to visually check results

In [11]:
display(df)

,File Name,Date Time,Leaf Area
0,IMG_0671.jpeg,2023:05:25 09:08:05,0.002756
1,IMG_0667.jpeg,2023:05:25 09:04:48,0.000456
2,IMG_0666.jpeg,2023:05:25 09:04:25,0.004903
3,IMG_0670.jpeg,2023:05:25 09:07:25,0.001588
4,IMG_0677.jpeg,2023:05:25 09:09:03,0.000894
5,IMG_0676.jpeg,2023:05:25 09:08:56,0.002177
6,IMG_0675.jpeg,2023:05:25 09:08:49,0.002216
7,IMG_0674.jpeg,2023:05:25 09:08:39,0.002071
8,IMG_0673.jpeg,2023:05:25 09:08:26,0.003269
9,IMG_0669.jpeg,2023:05:25 09:06:50,0.003128


## Save the dataframe

In [12]:
# set folder
folder = 'results/' # 'results/' folder is provided, but set any relative path desired

# set filename
filename = 'name.csv'

# save file
df.to_csv(folder + filename)

**Authors:**
Matthew Jenkins, 
Dylan Lenczewski-Jowers, 
Fabiola Chavez Lamas